In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai

In [ ]:
df = pd.read_csv("Post45_NEAData_Final.csv")

In [ ]:
df.head()

In [ ]:
# how many don't have gender listed?
df[df.gender == "unknown"]

In [ ]:
# how many don't have a race or ethnicity
print("No race", len(df[df.race_ethnicity == "uniden"]))
print("Race", len(df[df.race_ethnicity != "uniden"]))

In [ ]:
df.race_ethnicity.value_counts()

In [ ]:
df.race_ethnicity = df.race_ethnicity.replace("uniden / ","", regex = True)

In [ ]:
r_e = df.race_ethnicity.value_counts()
r_e

In [ ]:
race = r_e[1:10]
race.plot.bar(title = "Race of NEA Creative Writing Recipients")

In [ ]:
ba = df.ba.value_counts()[:11]
ba

In [ ]:
print("ba listed ",len(df) - df.ba.isna().sum())
print("ba listed ", ((len(df) - df.ba.isna().sum())/len(df))*100,"%")

In [ ]:
ba.plot.bar()

In [ ]:
ivies = ["Brown University", "Columbia University", "Cornell University", "Dartmouth College", 
"Harvard University", "University of Pennsylvania", "Princeton University", "Yale University"]

df["Ivy"] = df.ba.isin(ivies)
df.Ivy.value_counts()

In [ ]:
ivies = ["Brown University", "Columbia University", "Cornell University", "Dartmouth College", 
"Harvard University", "University of Pennsylvania", "Princeton University", "Yale University"]
tier_1_schools = [
    "Stanford University",
    "Massachusetts Institute of Technology (MIT)",
    "California Institute of Technology (Caltech)",
    "University of Chicago",
    "Duke University",
    "Northwestern University",
    "University of California, Berkeley (UC Berkeley)",
    "University of California, Los Angeles (UCLA)",
    "University of Michigan, Ann Arbor",
    "University of Texas at Austin",
    "Carnegie Mellon University",
    "University of Southern California (USC)",
    "Johns Hopkins University",
    "Georgia Institute of Technology (Georgia Tech)",
    "University of North Carolina at Chapel Hill"
]

tier_2_schools = [
    "University of Washington",
    "University of Wisconsin-Madison",
    "University of Illinois Urbana-Champaign",
    "University of Florida",
    "Pennsylvania State University (Penn State)",
    "Ohio State University",
    "Purdue University",
    "University of Maryland, College Park",
    "Texas A&M University",
    "University of Minnesota, Twin Cities",
    "University of Virginia",
    "University of California, San Diego (UCSD)",
    "University of California, Davis (UC Davis)",
    "Boston University",
    "University of Rochester",
    "Michigan State University",
    "Indiana University Bloomington",
    "University of Colorado Boulder",
    "University of Arizona",
    "University of Utah"
]

top_schools = ivies + tier_1_schools

# Define a function to fill the new column based on string matching
def fill_new_column(row):
    if row['ba'] in ivies:
        return "Ivy League"
    elif row['ba'] in tier_1_schools:
        return "Tier 1"
    elif row['ba'] in tier_2_schools:
        return "Tier 2"
    else:
        return "not_found"

# Apply the function to create a new column
df['Tier'] = df.apply(fill_new_column, axis=1)

df.Tier.value_counts()

In [ ]:
top_schools = ivies + tier_1_schools

# Define a function to fill the new column based on string matching
def fill_new_column_top(row):
    if row['ba'] in top_schools:
        return "Top School"
    else:
        return "Other"
        
# Apply the function to create a new column
df['Top_schools'] = df.apply(fill_new_column_top, axis=1)

top_schools = df.Top_schools.value_counts(normalize=True)
top_schools

In [ ]:
top_schools.plot.pie()

# NYTimes

In [2]:
nyt = pd.read_csv("nyt_full.tsv", sep = '\t')
nyt.head()

,year,week,rank,title_id,title,author
0,1931,1931-10-12,1,6477,THE TEN COMMANDMENTS,Warwick Deeping
1,1931,1931-10-12,2,1808,FINCHE'S FORTUNE,Mazo de la Roche
2,1931,1931-10-12,3,5304,THE GOOD EARTH,Pearl S. Buck
3,1931,1931-10-12,4,4038,SHADOWS ON THE ROCK,Willa Cather
4,1931,1931-10-12,5,3946,SCARMOUCHE THE KING MAKER,Rafael Sabatini


In [3]:
nyt.author.nunique()

2210

In [66]:
print(nyt.author)

0                        Warwick Deeping
1                       Mazo de la Roche
2                          Pearl S. Buck
3                           Willa Cather
4                        Rafael Sabatini
                      ...               
60381                             Halsey
60382                       Brit Bennett
60383                        Delia Owens
60384                    Fredrik Backman
60385    Clive Cussler and Boyd Morrison
Name: author, Length: 60386, dtype: object


In [83]:
nyt.author = nyt.author.replace("written and illustrated by ","", regex = True)

In [94]:
nyt['check'] = nyt.author.str.contains(' and | with | by', regex = True)
print(nyt.check.sum())
print(nyt.check.sum()/len(nyt))

2406
0.03984367237439142


In [95]:
nyt_no_multi = nyt[nyt.check != True]
nyt_multi = nyt[nyt.check == True]
print(len(nyt_no_multi))
print(len(nyt_multi))

57980
2406


In [101]:
nyt_multi['edited'] = nyt_multi.author.str.contains('edited', case = False, regex = True)
nyt_edited = nyt_multi[nyt_multi.edited == True]
len(nyt_edited)

/var/folders/p7/4d0x8hyj4mb4xypm89k57lrr0000gn/T/ipykernel_53150/2809466335.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyt_multi['edited'] = nyt_multi.author.str.contains('edited', case = False, regex = True)


93

In [104]:
print(len(nyt_edited.author.unique()))
nyt_edited.author.unique()

14


array(['Gideon Wyck, edited by Alexander Laing',
       'edited by Somerset Maugham', 'edited by Armin L. Robinson',
       'edited by Nicholas Meyer',
       'edited by William J. Bennett. Illustrated by Michael Hague',
       'Margaret Mitchell. Edited by Debra Freer',
       'edited by John Updike and Katrina Kenison',
       'edited by Joyce Reardon',
       'Hannah Crafts. Edited by Henry Louis Gates Jr',
       'J. R. R. Tolkien. Edited by Christopher Tolkien. Illustrated by Alan Lee',
       'J. R. R. Tolkien. Edited by Christopher Tolkien',
       'edited by Charlaine Harris and Toni L. P. Kelner',
       'edited by George R. R. Martin and Gardner Dozois',
       'Lucia Berlin, edited by Stephen Emerson'], dtype=object)

In [ ]:
# if there's an author with editor, remove editor, else treat editor as author 
nyt_edited['author_alt'] = 

if name[:10] == 'edited by ':
    remove 'edited by ' and transfer rest of string to the 'author_alt' column
if name[:10] != 'edited by '"
    

In [96]:
nyt_multi['three'] = nyt_multi.author.str.contains(',| with ', regex = True)
print(nyt_multi.three.sum())

nyt_two = nyt_multi[nyt_multi.three != True]
nyt_three_more = nyt_multi[nyt_multi.three == True]

181


/var/folders/p7/4d0x8hyj4mb4xypm89k57lrr0000gn/T/ipykernel_53150/476291792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyt_multi['three'] = nyt_multi.author.str.contains(',| with ', regex = True)


In [97]:
nyt_two.author.unique()

array(['Charles Nordhoff and James N. Hall',
       'Charles Nordloff and James Norman Hall',
       'Dennis Wheatley and J. H. Links', 'edited by Somerset Maugham',
       'Charles Nordhoff and James Norman Hall',
       'edited by Armin L. Robinson', 'Odell Shepard and Willard Shepard',
       '? by Isabel Bolton', 'Benedict Freedman and Nancy Freedman',
       'Joseph Auslander and Audrey Wurdemann',
       'James Street and James Childers',
       'Harnett T. Kane and Victor Leclerc', 'Marrijane and Joseph Hayes',
       'Dorothy Erskine and Patrick Dennis',
       'William J. Lederer and Eugene Burdick',
       'Arthur Quiller-Couch and Daphne du Maurier',
       'Fletcher Knebel and Charles W. Bailey II',
       'Eugene Burdick and Harvey Wheeler',
       'Terry Southern and Mason Hoffenberg', 'edited by Nicholas Meyer',
       'J.D. Gilman and John Clive', 'Marvin Kalb and Ted Koppel',
       'General Sir John Hackett and Other Top-ranking NATO Generals and Advisors',
       'Ar

In [ ]:
# sharing last name (i.e., Janet and Allan Ahlberg

# same author different spelling (i.e., James Norman Hall or James N. Hall)

# extraneous words (i.e. 'Edited by ', 'Illustrated by '


In [5]:
nyt_authors = nyt.author.unique().tolist()

# Because GPT can only do so many at once
import itertools as it

cnt = 0
auth50 = list(it.batched(nyt_authors, 50))
for item in auth50:
    cnt+=1
    c = str(cnt)
    with open("nytimes_auth/authors"+c+".txt", "w") as output:
        output.write(str(item))


# Use GPT to gather data about the authors

In [20]:
nyt_authors_short = nyt_authors[:5]
print(nyt_authors_short)
nyt_short = nyt[:5]
nyt_short

['Warwick Deeping', 'Mazo de la Roche', 'Pearl S. Buck', 'Willa Cather', 'Rafael Sabatini']


,year,week,rank,title_id,title,author
0,1931,1931-10-12,1,6477,THE TEN COMMANDMENTS,Warwick Deeping
1,1931,1931-10-12,2,1808,FINCHE'S FORTUNE,Mazo de la Roche
2,1931,1931-10-12,3,5304,THE GOOD EARTH,Pearl S. Buck
3,1931,1931-10-12,4,4038,SHADOWS ON THE ROCK,Willa Cather
4,1931,1931-10-12,5,3946,SCARMOUCHE THE KING MAKER,Rafael Sabatini


In [44]:
myapikey = open('my_gpt_apikey.txt').readline().rstrip()

client = openai.OpenAI(api_key = myapikey)

output = []
for author in nyt_authors_short:
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": """You are a literary historian. 
            You reply with only the requested information, not a complete sentence, and no headers. 
            You must include the name of the author in your response. If you do not know information, fill it in with None.
            If an author is still living, fill in the information with N/A.
            For example: Chimamanda Ngozi Adichie, 1977, N/A, Nigerian, Master's Degree, Johns Hopkins University"""},
            {
                "role": "user",
                "content": """What is the birth year, death year, race/ethnicity, highest level of education, and 
                college or university of """+author
            }
        ]
    )
    my_response = completion.choices[0].message.content
    response_list = my_response.split(',')
    output.append(response_list)



print(output)

[['Warwick Deeping', ' 1877', ' 1950', ' British', ' N/A', ' N/A'], ['Mazo de la Roche', ' 1879', ' 1961', ' White', " Bachelor's Degree", ' University of Toronto'], ['Pearl S. Buck', ' 1892', ' 1973', ' Caucasian', " Master's Degree", ' Cornell University'], ['Willa Cather', ' 1873', ' 1947', ' Caucasian', " Bachelor's Degree", ' University of Nebraska-Lincoln'], ['Rafael Sabatini', ' 1875', ' 1950', ' Italian', ' N/A', ' N/A']]


In [46]:
nyt_auth_demo = pd.DataFrame(output, columns = ['Name', 'Birth', 'Death', 'Race/Eth', 'Education', 'Institution'])
nyt_auth_demo

,Name,Birth,Death,Race/Eth,Education,Institution
0,Warwick Deeping,1877,1950,British,N/A,N/A
1,Mazo de la Roche,1879,1961,White,Bachelor's Degree,University of Toronto
2,Pearl S. Buck,1892,1973,Caucasian,Master's Degree,Cornell University
3,Willa Cather,1873,1947,Caucasian,Bachelor's Degree,University of Nebraska-Lincoln
4,Rafael Sabatini,1875,1950,Italian,N/A,N/A


In [ ]:
with open("../nytimes/authors.txt", "w") as output:
    output.write(str(nyt_authors))

In [ ]:
nyt.title.nunique()

In [ ]:
author_demo = pd.read_csv("../nytimes/authors_demographics.csv")

In [ ]:
author_demo.head()

In [ ]:
author_demo = author_demo.replace('NaN', pd.NA)

In [ ]:
author_demo.head()